# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.4.1

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
#def draw_landmarks(image, results):
 #   mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
  #  mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
   # mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
   # mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
#draw_landmarks(frame, results)

In [9]:
#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [10]:
#Making seperate array for pose landmarks
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test) 

In [11]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [12]:
results_test= extract_keypoints(results)

In [13]:
print("The extracted keypoints are:",results_test)

The extracted keypoints are: [ 0.62276137  0.51868165 -1.10160661 ...  0.10424758  0.38395619
 -0.07608117]


In [14]:
#468*3+33*4+21*3+21*3=1662
results_test.shape

(1662,)

In [15]:
print(results.face_landmarks.landmark)

[x: 0.6119589
y: 0.5980936
z: -0.027935967
, x: 0.6149573
y: 0.55017537
z: -0.054627683
, x: 0.61253405
y: 0.56397474
z: -0.02815097
, x: 0.6052754
y: 0.49713764
z: -0.042684104
, x: 0.6154569
y: 0.5343
z: -0.058325753
, x: 0.61504906
y: 0.5129077
z: -0.054578356
, x: 0.6129341
y: 0.46109876
z: -0.0277712
, x: 0.5357141
y: 0.4523417
z: 0.004712943
, x: 0.6125876
y: 0.42572978
z: -0.021803861
, x: 0.6131602
y: 0.40554398
z: -0.024326885
, x: 0.61309475
y: 0.3306975
z: -0.014276616
, x: 0.6116926
y: 0.60536325
z: -0.026411373
, x: 0.6111898
y: 0.6102308
z: -0.022864616
, x: 0.6106753
y: 0.61160606
z: -0.018193185
, x: 0.6110606
y: 0.61582947
z: -0.017452551
, x: 0.61120266
y: 0.6216909
z: -0.019059964
, x: 0.6111282
y: 0.62914
z: -0.021220272
, x: 0.6107886
y: 0.63714343
z: -0.019694222
, x: 0.60938305
y: 0.6519218
z: -0.009769297
, x: 0.6141801
y: 0.5577606
z: -0.049267083
, x: 0.6033008
y: 0.55555665
z: -0.03600539
, x: 0.48984966
y: 0.39401484
z: 0.047363047
, x: 0.56582713
y: 0.46705

In [16]:
print(results.pose_landmarks.landmark)

[x: 0.62276137
y: 0.51868165
z: -1.1016066
visibility: 0.9998526
, x: 0.65604883
y: 0.4488281
z: -1.0348248
visibility: 0.99976397
, x: 0.67716175
y: 0.45108163
z: -1.0347494
visibility: 0.9996461
, x: 0.6951782
y: 0.45414287
z: -1.0346023
visibility: 0.9996299
, x: 0.58238536
y: 0.4469568
z: -1.0133073
visibility: 0.9998155
, x: 0.5595034
y: 0.44727033
z: -1.0125585
visibility: 0.9998099
, x: 0.5422441
y: 0.44928512
z: -1.0126355
visibility: 0.9998759
, x: 0.71061957
y: 0.49398434
z: -0.59479105
visibility: 0.9995818
, x: 0.51936287
y: 0.4883117
z: -0.46446466
visibility: 0.99985844
, x: 0.6643158
y: 0.60158664
z: -0.9467368
visibility: 0.99986064
, x: 0.57160366
y: 0.59976876
z: -0.91120845
visibility: 0.99989444
, x: 0.8789902
y: 0.89104754
z: -0.32258144
visibility: 0.9919225
, x: 0.3709437
y: 0.86481446
z: -0.25543013
visibility: 0.99908066
, x: 1.0226814
y: 1.3761107
z: -0.4788742
visibility: 0.19788824
, x: 0.123270534
y: 1.2576164
z: -0.89577246
visibility: 0.9660196
, x: 0.969

In [17]:
#print(results.left_hand_landmarks.landmark)

In [18]:
#print(results.right_hand_landmarks.landmark)

In [19]:
np.save("Extracted keypoints",results_test)

## 4. Setup folder

In [20]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('DATASET') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'Thumbsdown','Thumbsup'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [21]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

'for action in actions: \n    for sequence in range(no_sequences):\n        try: \n            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))\n        except:\n            pass'

In [22]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

"cap = cv2.VideoCapture(0)\n# Set mediapipe model \nwith mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:\n    \n    # NEW LOOP\n    # Loop through actions\n    for action in actions:\n        # Loop through sequences aka videos\n        for sequence in range(no_sequences):\n            # Loop through video length aka sequence length\n            for frame_num in range(sequence_length):\n\n                # Read feed\n                ret, frame = cap.read()\n\n                # Make detections\n                image, results = mediapipe_detection(frame, holistic)\n#                 print(results)\n\n                # Draw landmarks\n                draw_styled_landmarks(image, results)\n                \n                # NEW Apply wait logic\n                if frame_num == 0: \n                    cv2.putText(image, 'STARTING COLLECTION', (120,200), \n                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)\n 

# 6. Preprocess Data and Create Labels and Features

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [24]:
label_map = {label:num for num, label in enumerate(actions)}

In [25]:
label_map

{'hello': 0, 'thanks': 1, 'Thumbsdown': 2, 'Thumbsup': 3}

In [26]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [27]:
np.array(sequences).shape
#120 no.of vieos
#30 no.of frames in each video
#1662 keypoints in each frame

(120, 30, 1662)

In [28]:
np.array(labels).shape #this says the label(action number) of each video

(120,)

In [29]:
x=np.array(sequences)

In [30]:
x.shape

(120, 30, 1662)

In [31]:
y=to_categorical(labels).astype(int)

In [32]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [33]:
y.shape

(120, 4)

In [34]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [35]:
x_train.shape

(96, 30, 1662)

In [36]:
y_train.shape

(96, 4)

# 7. Build and Train LSTM Neural Network


In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [38]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir) 
#this Logs area gives the result in graph format

In [39]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) #softmax says the max.probability of an action

"model = Sequential()\nmodel.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))\nmodel.add(LSTM(128, return_sequences=True, activation='relu'))\nmodel.add(LSTM(128, return_sequences=False, activation='relu'))\nmodel.add(Dense(64, activation='relu'))\nmodel.add(Dense(32, activation='relu'))\nmodel.add(Dense(actions.shape[0], activation='softmax')) #softmax says the max.probability of an action"

In [40]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

In [41]:
model.fit(x_train, y_train, epochs=200, callbacks=[tb_callback])

In [42]:
model.save("model_output.h5")

In [43]:
model = tf.keras.models.load_model('model_output.h5')

# 8.Make predictions

In [44]:
res=model.predict(x_test)

1/1 [==============================] - 2s 2s/step


In [45]:
res #this says the probability of each action

array([[6.97082723e-04, 9.69255626e-01, 7.31244276e-04, 2.93160137e-02],
       [3.96219594e-03, 6.35446787e-01, 1.64191274e-03, 3.58949155e-01],
       [4.88558441e-01, 1.52322915e-04, 2.58718550e-01, 2.52570659e-01],
       [6.05198264e-04, 9.69651699e-01, 6.15936471e-04, 2.91271508e-02],
       [1.06253270e-02, 1.17899854e-05, 9.71560538e-01, 1.78022813e-02],
       [4.77742076e-01, 2.82085093e-04, 2.09822521e-01, 3.12153339e-01],
       [4.66616571e-01, 4.54865949e-04, 1.50315210e-01, 3.82613331e-01],
       [8.69103819e-02, 1.60444546e-02, 1.52613064e-02, 8.81783903e-01],
       [2.01403052e-01, 5.86536014e-03, 3.20921540e-02, 7.60639429e-01],
       [4.58629668e-01, 1.63399804e-04, 3.14396620e-01, 2.26810321e-01],
       [1.46436796e-01, 5.37275264e-05, 7.77869821e-01, 7.56396651e-02],
       [6.36713638e-04, 9.58398581e-01, 5.57854888e-04, 4.04068455e-02],
       [4.17929769e-01, 5.65911250e-05, 4.35511500e-01, 1.46502107e-01],
       [4.80167240e-01, 2.60965811e-04, 2.06965357e

In [46]:
actions[np.argmax(res[2])]

'hello'

In [47]:
actions[np.argmax(y_test[2])]

'hello'

In [48]:
model.save('actions.keras') #saves the predictions of testdata

# 9.Confusion matrix

In [49]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [50]:
y_pred = model.predict(x_test)

1/1 [==============================] - 0s 104ms/step


In [51]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(y_pred, axis=1).tolist()

In [52]:
multilabel_confusion_matrix(ytrue, yhat).shape

(4, 2, 2)

In [53]:
multilabel_confusion_matrix(ytrue, yhat) #this says the confusion matrix for each action

array([[[15,  3],
        [ 1,  5]],

       [[15,  0],
        [ 0,  9]],

       [[21,  1],
        [ 0,  2]],

       [[17,  0],
        [ 3,  4]]], dtype=int64)

In [54]:
test_set=accuracy_score(ytrue,yhat)

In [55]:
test_set

0.8333333333333334

# Test in real time

In [56]:
colors = [(245,117,16), (117,245,16), (16,117,245),(20,120,250)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [72]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.455555555555

cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Visualiation logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Visualiation probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [58]:
cap.release()
cv2.destroyAllWindows()

In [59]:
model.save("Final.h5")

C:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [60]:
#plt.figure(figsize=(18,18))
#plt.imshow(prob_viz(res, actions, image, colors))